# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [4]:
# How many features have missing values?
df.isna().sum() # None

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

The Business Objective of this task is to accurately predict whether a client has subscribed to a term deposit based on attributes from marketing campaigns conducted by a Portuguese bank.

In [9]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [13]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

In [7]:
transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), ['job', 'marital', 'education', 'default', 'housing',
                                                                          'loan', 'contact', 'month', 'day_of_week', 'poutcome']),
                                     remainder = StandardScaler())

In [25]:
X = df.drop(labels = ['y'], axis = 1)
y = df['y']

In [26]:
X_new = transformer.fit_transform(X)
y_new = np.where(y == 'no', 0, 1)

In [28]:
X.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [15]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.3, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [19]:
#baseline model = value counts of target variable
df['y'].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
lgr = LogisticRegression()

lgr.fit(X_train, y_train)

C:\Users\skama\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Problem 9: Score the Model

What is the accuracy of your model?

In [29]:
lgr.score(X_test, y_test)

0.9121955167111758

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import time

In [40]:
models = [
    #Logistic Regression
    Pipeline([
        ('lgr', LogisticRegression(max_iter = 5000))
    ]),
    
    #Decision Tree
    Pipeline([
        ('tree', DecisionTreeClassifier())
    ]),
    
    #SVC
    Pipeline([
        ('svc', SVC())
    ]),
    
    #KNN
    Pipeline([
        ('knn', KNeighborsClassifier())
    ]),
]

In [32]:
model_name = []
train_time = []
train_acc = []
test_acc = []

for model in models:
    #get model name
    pipe = model.named_steps
    model_name.append(list(pipe.keys())[0])
    
    #get starting time
    start_time = time.time()
    
    #fit model
    model.fit(X_train, y_train)
    
    #get ending time
    train_time.append(time.time() - start_time)
    
    #score model - train
    score = model.score(X_train, y_train)
    train_acc.append(score)
    
    #score model - test
    score = model.score(X_test, y_test)
    test_acc.append(score)

C:\Users\skama\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\skama\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid thi

In [34]:
results = {'Model': model_name, 'Train Time': train_time, 'Train Accuracy': train_acc, 'Test Accuracy': test_acc}

results = pd.DataFrame(results)

results

,Model,Train Time,Train Accuracy,Test Accuracy
0,lgr,0.604138,0.911727,0.912196
1,tree,0.415207,1.000000,0.887675
2,svc,24.634876,0.923346,0.911548
3,knn,0.000000,0.927266,0.901109


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [38]:
from sklearn.model_selection import GridSearchCV

In [35]:
#Set up params for hyperparameter tuning
param_grids = [
    #Logistic Regression
    {'lgr__C': [0.1, 0.2, 0.3], 'lgr__penalty': ['l1', 'l2'], 'lgr__solver': ['liblinear', 'saga']},
    #Decision Tree
    {'tree__max_depth': [100, 200, 300], 'tree__min_samples_split': [20, 40, 60]},
    #SVM
    {'svc__C': [0.1, 0.2, 0.3], 'svc__kernel': ['linear', 'sigmoid']},
    #KNN
    {'knn__n_neighbors': [1, 5, 10], 'knn__weights': ['uniform', 'distance']}
]

In [36]:
# Set up a dictionary to store the results
hyper_results = {
    'model': [
        'Logistic Regression',
        'Decision Tree', 
        'SVM',
        'KNN'
    ],
    'best_params': ['' for _ in range(6)],
    'best_score': ['' for _ in range(6)],
    'mean_fit_time': ['' for _ in range(6)]
}

In [41]:
for i, (model, param_grid) in enumerate(zip(models, param_grids)):
    print((model, param_grid))
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    hyper_results['best_params'][i] = str(grid_search.best_params_)
    hyper_results['best_score'][i] = grid_search.best_score_
    hyper_results['mean_fit_time'][i] = grid_search.cv_results_['mean_fit_time'][grid_search.best_index_]

    print(f"  Best params: {hyper_results['best_params'][i]}")
    print(f"  Best cross-validation score: {hyper_results['best_score'][i]:.4f}")
    print(f"  Mean fit time: {hyper_results['mean_fit_time'][i]:.2f}s")
    print()

(Pipeline(steps=[('lgr', LogisticRegression(max_iter=5000))]), {'lgr__C': [0.1, 0.2, 0.3], 'lgr__penalty': ['l1', 'l2'], 'lgr__solver': ['liblinear', 'saga']})
  Best params: {'lgr__C': 0.3, 'lgr__penalty': 'l1', 'lgr__solver': 'saga'}
  Best cross-validation score: 0.9110
  Mean fit time: 4.87s

(Pipeline(steps=[('tree', DecisionTreeClassifier())]), {'tree__max_depth': [100, 200, 300], 'tree__min_samples_split': [20, 40, 60]})
  Best params: {'tree__max_depth': 300, 'tree__min_samples_split': 60}
  Best cross-validation score: 0.9056
  Mean fit time: 0.31s

(Pipeline(steps=[('svc', SVC())]), {'svc__C': [0.1, 0.2, 0.3], 'svc__kernel': ['linear', 'sigmoid']})
  Best params: {'svc__C': 0.3, 'svc__kernel': 'linear'}
  Best cross-validation score: 0.9026
  Mean fit time: 14.15s

(Pipeline(steps=[('knn', KNeighborsClassifier())]), {'knn__n_neighbors': [1, 5, 10], 'knn__weights': ['uniform', 'distance']})


C:\Users\skama\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\skama\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

  Best params: {'knn__n_neighbors': 10, 'knn__weights': 'uniform'}
  Best cross-validation score: 0.9038
  Mean fit time: 0.00s



ValueError: All arrays must be of the same length

In [67]:
# Create a Pandas DataFrame from the results
hyper_results_df = pd.DataFrame(hyper_results).set_index('model')

# Print the results
hyper_results_df

,best_params,best_score,mean_fit_time
model,,,
Logistic Regression,"{'lgr__C': 0.3, 'lgr__penalty': 'l1', 'lgr__so...",0.911033,4.873480
Decision Tree,"{'tree__max_depth': 300, 'tree__min_samples_sp...",0.905622,0.308188
SVM,"{'svc__C': 0.3, 'svc__kernel': 'linear'}",0.902640,14.148325
KNN,"{'knn__n_neighbors': 10, 'knn__weights': 'unif...",0.903784,0.003209


In [66]:
#Get test accuracy
models = [
    #Logistic Regression
    Pipeline([
        ('lgr', LogisticRegression(max_iter = 5000, C = .3, penalty = 'l1', solver = 'saga'))
    ]),
    
    #Decision Tree
    Pipeline([
        ('tree', DecisionTreeClassifier(max_depth = 300, min_samples_split = 60))
    ]),
    
    #SVC
    Pipeline([
        ('svc', SVC(C = .3, kernel = 'linear'))
    ]),
    
    #KNN
    Pipeline([
        ('knn', KNeighborsClassifier(n_neighbors = 10, weights = 'uniform'))
    ]),
]

In [68]:
for model in models:
    #fit model
    model.fit(X_train, y_train)
    
    #score model - train
    score = model.score(X_train, y_train)
    print(f"Train Accuracy: {score}")

    
    #score model - test
    score = model.score(X_test, y_test)
    print(f"Test Accuracy: {score}")

Train Accuracy: 0.9116922756754882
Test Accuracy: 0.9124382940843246
Train Accuracy: 0.9333356456591863
Test Accuracy: 0.905964230800356
Train Accuracy: 0.9022926710832091
Test Accuracy: 0.9030509023225702


C:\Users\skama\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Train Accuracy: 0.9150567097915439


C:\Users\skama\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Test Accuracy: 0.9064497855466537


##### Questions